![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)

Find this notebook in https://colab.research.google
.com/github/ricardokleinklein/NLP_GenMods/blob/main/Tacotron2.ipynb

# Modelos Generativos

## Tacotron2 - Audio

Creado por *Ricardo Kleinlein* para [Saturdays.AI](https://saturdays.ai/).

Disponible bajo una licencia [Creative Commons](https://creativecommons.org/licenses/by/4.0/).

---

## Sobre el uso de Jupyter Notebooks

Este notebook ha sido implementado en Python, pero para su ejecución no es
necesario conocer el lenguaje en profundidad. Solamente se debe ejecutar cada
una de las celdas, teniendo en cuenta que hay que ejecutar una celda a la vez
y secuencialmente, tal y como figuran en orden de aparición.

Para ejecutar cada celda pulse en el botón ▶ en la esquina superior izquierda
de cada celda. Mientras se esté ejecutando ese fragmento de código,
el botón estará girando. En caso de querer detener dicha ejecución, pulse
nuevamente sobre este botón mientras gira y la ejecución se detendrá. En caso
de que la celda tenga alguna salida (texto, gráficos, etc) será mostrada
justo después de esta y antes de mostrar la siguiente celda. El notebook
estará guiado con todas las explicaciones necesarias, además irá acompañado
por comentarios en el código para facilitar su lectura.

En caso de tener alguna duda, anótela. Dedicaremos un tiempo a plantear y
resolver la mayoría delas dudas que puedan aparecer.


## Objetivo del notebook

Implementar, descargar y utilizar un modelo de Estado del Arte (Tacotron2)
de Text-To-Speech Synthesis (TTS).

## Sobre el modelo

A la hora de generar una voz sintética, existen una serie de factores que se
 engloban dentro de lo que denominamos "prosodia", que constituyen algunos
 factores especialmente peliagudos de modelar por sistemas automáticos: el
 ritmo, el énfasis o la entonación. Entre otros atributos físicos, son estos
  factores los que hacen una voz más reconocible que otra en muchos casos.

Hemos visto en las diapositivas un poco sobre el modelo Wavenet de
generación de habla natural. En ese caso, el modelo era un modelo
autorregresivo, esto es, que empleaba predicciones anteriores para elaborar
futuros puntos de la muestra.

El modelo Tacotron original [[paper](https://arxiv.org/abs/1703.10135)] empleaba como componente fundamental
 Wavenet a la hora de construir habla. Sin embargo, dicho modelo es muy
 lento a la hora de generar, puesto que tiene que ver muy atrás en el tiempo
  para generar cada punto de la muestra.

Por ello, Tacotron2 [[paper](https://arxiv.org/abs/1712.05884)] construye sobre esta idea, y propone una
solución de compromiso donde sacrifica parte de la "personalidad" de la voz
por eficiencia en la generación. Si bien Wavenet entraba dentro de la
familia de modelos autorregresivos, Tacotron2 se enmarca dentro de las
estrategias "flow-density".

En la imagen inferior se muestra un diagrama de las partes que componen este
 sistema de síntesis de voz natural.

![tacotron2-diagram](./assets/tacotron2_diagram.png)

El modelo complementario WaveGlow es un modelo que ha aprendido a generar
espectrogramas a partir de texto. Mediante la combinación de Tacotron2 con
WaveGlow, el texto nuevo que escribamos como input podrá ser interpretado
como habla natural, y se generará en formato de audio.

Se podrían modificar aspectos de la voz resultante incorporando información
adicional a diferentes niveles dentro del modelo, pero en este ejercicio nos
 vamos a centrar en cargar el modelo y generar nuestros propios audios.


## Instalar las librerías necesarias

In [1]:
%%bash
pip install numpy scipy librosa unidecode inflect librosa
apt-get update
apt-get install -y libsndfile1

  Using cached resampy-0.2.2.tar.gz (323 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached audioread-2.1.9.tar.gz (377 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached SoundFile-0.10.3.post1-py2.py3-none-any.whl (21 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Created wheel for audioread: filename=audioread-2.1.9-py3-none-any.whl size=23143 sha256=9bda96d05c6d523caf68b2605285ecfeadbce02dd6a7021601a25287669d7171
  Stored in directory: /home/ricardokleinlein/.cache/pip/wheels/ba/7b/eb/213741ccc0678f63e346ab8dff10495995ca3f426af87b8d88
  Created wheel for resampy: filename=resampy-0.2.2-py3-none-any.whl size=320719 sha256=d64093fa0a5cbd07fab58b30dffcf60aa08c1cd83bdcbf7156e5e0f9ba5af1d9
  Stored in directory: /home/ricardokleinlein/.cache/pip/wheels/a0/18/0a/8ad18a597d8333a142c9789338a96a6208f1198d290ece356c
Successfully built au

E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


CalledProcessError: Command 'b'pip install numpy scipy librosa unidecode inflect librosa\napt-get update\napt-get install -y libsndfile1\n\n'' returned non-zero exit status 100.

## Importar los modelos pre-entrenados

Estos modelos ocupan mucho espacio en memoria, pero sus tiempos de
entrenamiento son aún peores, y requieren de una infraestructura avanzada
para poder entrenarlos en plazos de tiempo razonables. Desde luego, exceden
en mucho las capacidades de la mayoría de nuestros ordenadores, o del
servidor default que Colab nos proporciona.

Afortunadamente, NVIDIA proporciona un servidor desde el que descargar un
modelo completamente preentrenado.

### Tacotron2

Esta versión de Tacotron2 es casi idéntica en arquitectura al original como
aparece publicado en el paper, con modificaciones mínimas en algunas capas.
Ha sido entrenado en la base de datos [LJSpeech](https://keithito.com/LJ-Speech-Dataset/), la cual constituye una
de las referencias principales a la hora de entrenar modelos de síntesis de
voz. Probablemente la otra mayor base de datos a tal efecto sea [VCTK](https://datashare.ed.ac.uk/handle/10283/2950),
desarrollada por Junichi Yamagishi en Edimburgo, con quién trabajé en Tokyo.

LJSpeech consta de ...

In [2]:
from typing import Tuple
from IPython.display import Audio
import torch

TacotronModel = Tuple[torch.nn.Module, torch.nn.Module]

tacotron2 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub',
                           'nvidia_tacotron2', model_math='fp16')
tacotron2 = tacotron2.to('cuda')
tacotron2.eval()

Downloading: "https://github.com/NVIDIA/DeepLearningExamples/archive/torchhub.zip" to /home/ricardokleinlein/.cache/torch/hub/torchhub.zip


Tacotron2(
  (embedding): Embedding(148, 512)
  (encoder): Encoder(
    (convolutions): ModuleList(
      (0): Sequential(
        (0): ConvNorm(
          (conv): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        )
        (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): Sequential(
        (0): ConvNorm(
          (conv): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        )
        (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (2): Sequential(
        (0): ConvNorm(
          (conv): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        )
        (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (lstm): LSTM(512, 256, batch_first=True, bidirectional=True)
  )
  (decoder): Decoder(
    (prenet): Prenet(
      (layers): ModuleList(
        (0): LinearNorm(
          (lin

Podemos repasar las líneas desplegadas para comprobar, junto con el diagrama
 mostrado al inicio, que la arquitectura es correcta.

### WaveGlow

En nuestro ejemplo, WaveGlow juega el rol de un *vocoder*, una herramienta
que convierte una codificación numérica del habla en sonidos audibles.

In [ ]:
waveglow = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_waveglow', model_math='fp16')
waveglow = waveglow.remove_weightnorm(waveglow)
waveglow = waveglow.to('cuda')
waveglow.eval()

En este momento ya estamos preparados para sintetizar audio. Por comodidad,
vamos a agrupar una serie de operaciones dedicadas a preprocesar el input
con que alimentaremos al modelo:

In [ ]:
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub',
                       'nvidia_tts_utils')

def synthesize(text: str, model: TacotronModel):
    """Adjust input text length by padding, and feed to model.

    :param text: Uttered speech.
    :param model: Tuple with instances of (Tacotron, WaveGlow).
    :return:
        numpy.ndarray with utterance.
    """
    sequences, lengths = utils.prepare_input_sequence([text])
    with torch.no_grad():
        mel, _, _ = model[0].infer(sequences, lengths)
        audio = model[1].infer(mel)
    return audio[0].data.cpu().numpy()


## Playground

Ahora solo resta escribir una cadena de texto (en inglés para obtener
mejores resultados) y escuchar cuál es el resultado.

In [ ]:
text = "Isn't Machine Learning something absolutely fabulous?"
signal = synthesize(text, (tacotron2, waveglow))
Audio(signal, rate=22050)